In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
def encode_symbols(symbols):
    # Make unique characters
    symbols = set(symbols)
    # Sort for logical ordering scheme
    symbols = sorted(symbols)
    sym_to_code = {sym:code for code,sym in enumerate(symbols)}
    code_to_sym = {code:sym for code,sym in enumerate(symbols)}
    return sym_to_code, code_to_sym

In [3]:
text = open('sherlock.txt', 'r').read()
# Better to use word2vec
char_to_code, code_to_char = encode_symbols(text)
# Constants
vocab_size = len(char_to_code)
n_hidden = 256
n_epochs = 1
seq_length = 10

# Init
x = tf.placeholder('float', [None, seq_length, ])
y = tf.placeholder('float')
hy = {
    'weights': tf.Variable(tf.random_normal([n_hidden, vocab_size])),
    'biases': tf.Variable(tf.random_normal([vocab_size]))
}

In [4]:
def RNN(x):
    x = tf.reshape(x, [-1, seq_length])
    
    # Create sequence
    x = tf.split(x, seq_length, 1)
    
    rnn_cell = rnn.BasicLSTMCell(n_hidden)
    
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], hy['weights'] + hy['biases'])

In [5]:
def train_rnn(x):
    pred = RNN(x)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
    optimizer = tf.train.RMSPropOptimizer(learning_rate = 0.01).minimize(loss)
    
    with tf.Session() as sess:
        #sess.run(tf.initialize_all_variables())
        global_variables_initializer()
        sess.run()
        for epoch in range(n_epochs):
            p = 0
            epoch_loss = 0
            while p + seq_length + 1 < len(text):
                train_seq = [char_to_code[ch] for ch in text[p:p+seq_length]]
                label_seq = [char_to_code[ch] for ch in text[p+1:p+seq_length+1]]
                _, c = sess.run([optimizer, loss], feed_dict={x: train_seq, y: label_seq})
                epoch_loss += c
                p += seq_length
                
            print("{} epochs completed out of {}".format(epoch+1, num_epochs))
            correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
            print("Accuracy: {}".format(accuracy))

In [9]:
tf.reset_default_graph()
# Init
x = tf.placeholder('float', [None, seq_length, seq_length])
y = tf.placeholder('float')
train_rnn(x)

ValueError: Tensor("add_2:0", shape=(256, 81), dtype=float32) must be from the same graph as Tensor("rnn/rnn/basic_lstm_cell/mul_29:0", shape=(?, 256), dtype=float32).